<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#setup" data-toc-modified-id="setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>setup</a></span></li><li><span><a href="#测试简单函数" data-toc-modified-id="测试简单函数-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>测试简单函数</a></span><ul class="toc-item"><li><span><a href="#翻译" data-toc-modified-id="翻译-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>翻译</a></span></li><li><span><a href="#删除-%-%-[]-内的内容" data-toc-modified-id="删除-%-%-[]-内的内容-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>删除 % % [] 内的内容</a></span></li><li><span><a href="#测试-spacy" data-toc-modified-id="测试-spacy-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>测试 spacy</a></span></li><li><span><a href="#nltk" data-toc-modified-id="nltk-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>nltk</a></span></li><li><span><a href="#生成训练数据" data-toc-modified-id="生成训练数据-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>生成训练数据</a></span></li><li><span><a href="#近似向量查找" data-toc-modified-id="近似向量查找-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>近似向量查找</a></span></li></ul></li><li><span><a href="#调试函数" data-toc-modified-id="调试函数-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>调试函数</a></span><ul class="toc-item"><li><span><a href="#集成测试" data-toc-modified-id="集成测试-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>集成测试</a></span></li><li><span><a href="#单元测试" data-toc-modified-id="单元测试-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>单元测试</a></span></li></ul></li><li><span><a href="#收集专有词汇" data-toc-modified-id="收集专有词汇-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>收集专有词汇</a></span></li></ul></div>

## setup

In [69]:
# %load -r 1-17 G

import os
import re
import sys
import jieba
import pandas as pd
import spacy
from stemming.porter2 import stem
from googletrans import Translator
from multiprocessing import Pool

try:
    from ..nmt_common_fn import *
except:
    sys.path.append('/aiml/data/DevMind/06.SRC/01.TestCaseBot/02.TestAWBot/01.NMT_Train')
    from nmt.nmt_common_fn import *


## 测试简单函数

### 翻译

In [440]:
from googletrans import Translator
translator = Translator()
translator.translate("去配置二层组播实例基础业务").text

'Configure basic services of Layer 2 multicast instances'

### 删除 % % [] 内的内容

In [215]:
import re
reg_bracket = re.compile('\[.*?\]|\%.*?\%')
reg_bracket.sub(' ', 'display pim[ %ver%]')

'display pim '

### 测试 spacy

In [312]:
# Install: pip install spacy && python -m spacy download en
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load('en')

# Process whole documents
text = u'the fries were gross'
doc = nlp(text)

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

# Determine semantic similarities
doc1 = nlp(u'the fries were gross')
doc2 = nlp(u'worst fries ever')
doc1.similarity(doc2)

0.5829033582528316

### nltk

词形还原

In [415]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
wordnet_lemmatizer.lemmatize('dogs')
wordnet_lemmatizer.lemmatize('snooping', pos='v')
wordnet_lemmatizer.lemmatize('snooping')

'dog'

'snoop'

'snooping'

词干提取，参考[http://www.zmonster.me/2016/01/21/lemmatization-survey.html]

In [401]:
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
porter_stemmer.stem('dogs')
porter_stemmer.stem('crying')

'dog'

'cri'

### 生成训练数据

In [449]:
import pandas as pd
df = pd.read_csv('/tmp/clean_data.csv')
with open('/tmp/clean_data.txt', 'w') as f:
    f.writelines([x + '\n' for x in df['train_data'].tolist() if isinstance(x, str)])

### 近似向量查找

## 调试函数

In [471]:
# %load -s Deal_Str,deal_with_pd,read_all_csv /aiml/data/DevMind/06.SRC/01.TestCaseBot/02.TestAWBot/01.NMT_Train/nmt/preprocess/clean_data.py
class Deal_Str(object):
    def __init__(self):
        self.translator = Translator()
        self.regx = re.compile('\[.*?\]|\%.*?\%')
        # wordnet_lemmatizer = WordNetLemmatizer()
        if os.path.exists('wordninja_words.txt'):
            jieba.load_userdict('wordninja_words.txt')
        elif os.path.exists('./nmt/preprocess/wordninja_words.txt'):
            jieba.load_userdict('./nmt/preprocess/wordninja_words.txt')
        else:
            jieba.load_userdict('/aiml/data/DevMind/06.SRC/01.TestCaseBot/02.TestAWBot/01.NMT_Train/nmt/preprocess/wordninja_words.txt')
        print("Load Jieba Dict")

    @staticmethod
    @decorator_trace_call
    def method_to_des(mth):
        """
        将给定的字符串依据驼峰命名/下划线进行切分
        """

        mth = re.sub("[^A-Za-z0-9?=]", " ", mth)
        if not mth.islower():
            mth_arr = []
            for word in mth.split(" "):
                if word.islower():
                    mth_arr.append(word)
                    continue

                tmp_word = ""
                compare_len = len(word) - 1
                for i, alp in enumerate(word):
                    tmp_word += alp
                    if i == compare_len:
                        mth_arr.append(tmp_word)
                        break
                    before_alp_lower = word[i].islower()
                    after_alp_upper = word[i + 1].isupper()
                    before_alp_digit = word[i].isdigit()
                    if (i>0 and before_alp_digit and word[i-1].isupper() != after_alp_upper) \
                        or (before_alp_lower and after_alp_upper):
                        mth_arr.append(tmp_word)
                        tmp_word = ""
            mth_result = ' '.join(mth_arr)
        else:
            mth_result = mth
        mth_result = ' '.join([x.strip().lower() for x in mth_result.split() if "" != x])
        return mth_result

    @staticmethod
    @decorator_trace_call
    def rm_new_line_toke(input_str):
        """
        删除多余的符号
        """

        return ' '.join([x for x in input_str.split('\\r\\n') if x != ''])

    @decorator_trace_call
    def recreate_session(self):
        self.translator = Translator()

    @decorator_trace_call
    def deal_english_like_str(self, input_str):
        """
        处理类似英文的输入
        """

        if isinstance(input_str, str):
            # 删除 % % [] 内的内容
            ret_str = self.regx.sub(' ', input_str)
            ret_str = ' '.join([x.strip() for x in ret_str.split() if "" != x])
            # 删除 \r\n 然后根据函数命名的特征完成分词
            ret_str = method_to_des(rm_new_line_toke(ret_str))
            # 完成词形还原 词干提取
            # ret_str = ' '.join([wordnet_lemmatizer.lemmatize(x) for x in ret_str.split()])
            # 通过jieba将连续的包含多个词的字串分开
            ret_str = ' '.join([x for x in jieba.cut(ret_str) if "" != x and not x.isspace()])
            return ret_str
        else:
            return input_str

    @decorator_trace_call
    def deal_chinese_like_str(self, input_str):
        """
        处理含义中文的输入
        """

        def has_chinese(input_str):
            for x in input_str:
                if ord(x) <= 0x9FA5 and ord(x) >= 0x4E00:
                    return True
            return False

        if isinstance(input_str, str) and has_chinese(input_str):
            # 删除 \r\n
            ret_str = rm_new_line_toke(input_str)
            # 通过翻译接口将中文转换为英文
            ret_str = self.translator.translate(ret_str).text
            return ret_str
        else:
            return input_str

def deal_with_pd(df_idx):
    df = df_idx[0]
    ret_idx = df_idx[1]
    deal_str = Deal_Str()
    new_aw_des = []
    new_attr_aw_keyword = []
    new_aw_cmd = []
    new_attr_belongingclass = []
    new_attr_aw = []
    train_data = []
    process_idx = 0
    for idx, row in df.iterrows():
        process_idx += 1
        if process_idx % 500 == 0:
            deal_str.recreate_session()
        aw_des = deal_str.deal_english_like_str(deal_str.deal_chinese_like_str(row['aw_des']))
        attr_aw_keyword = deal_str.deal_english_like_str(row['attr_aw_keyword'])
        aw_cmd = deal_str.deal_english_like_str(row['aw_cmd'])
        attr_belongingclass = deal_str.deal_english_like_str(row['attr_belongingclass'])
        attr_aw = deal_str.deal_english_like_str(row['attr_aw'])
        single_train_data = ' '.join([x for x in [aw_des, attr_aw_keyword, aw_cmd, attr_belongingclass, attr_aw] if isinstance(x, str)])

        new_aw_des.append(aw_des)
        new_attr_aw_keyword.append(attr_aw_keyword)
        new_aw_cmd.append(aw_cmd)
        new_attr_belongingclass.append(attr_belongingclass)
        new_attr_aw.append(attr_aw)
        train_data.append(single_train_data)

    df = df.assign(new_aw_des=new_aw_des)
    df = df.assign(new_attr_aw_keyword=new_attr_aw_keyword)
    df = df.assign(new_aw_cmd=new_aw_cmd)
    df = df.assign(new_attr_belongingclass=new_attr_belongingclass)
    df = df.assign(new_attr_aw=new_attr_aw)
    df = df.assign(train_data=train_data)
    df = df.loc[:, ['_id', 'aw_des', 'new_aw_des', 'attr_aw_keyword', 'new_attr_aw_keyword',
                    'aw_cmd', 'new_aw_cmd', 'attr_belongingclass', 'new_attr_belongingclass',
                    'attr_aw', 'new_attr_aw', 'train_data']]
    return ret_idx, df

@decorator_trace_call
def read_all_csv(csv_list):
    """
    读取多个 CSV 文件，选取需要的字段然后合并
    """

    tmp_df_list = []
    for csv in csv_list:
        if os.path.exists(csv):
            tmp_df = pd.read_csv(csv)
            tmp_df = tmp_df.loc[:, ['_id', 'aw_des', 'attr_aw_keyword', 'aw_cmd', 'attr_belongingclass', 'attr_aw']]
            tmp_df_list.append(tmp_df)
            print("Read CSV file: %s" % (csv))
        else:
            print("Cannot find CSV file: %s" % (csv))
    return pd.concat(tmp_df_list)


### 集成测试

In [468]:
# %load -r 170- /aiml/data/DevMind/06.SRC/01.TestCaseBot/02.TestAWBot/01.NMT_Train/nmt/preprocess/clean_data.py
set_debug_flag()
df = read_all_csv(["/aiml/data/NoS_AW_Script/aw_baw.csv", "/aiml/data/NoS_AW_Script/aw_caw.csv"])
process_num = 40
line_per_process = 200

# TODO 调试代码
df = df[100: 200]

df_list = []
for idx in range(0, len(df), line_per_process):
    df_list.append([df[idx: idx+line_per_process], idx])

pool = Pool(process_num)
df_list = pool.map(deal_with_pd, df_list)
df_sorted = [x[1] for x in sorted(df_list, key=lambda x: x[0])]
pd.concat(df_sorted).to_csv('/tmp/first_tmp2.csv')
print(all(pd.concat(df_sorted)['_id'] == df['_id']))


Read CSV file: /aiml/data/NoS_AW_Script/aw_baw.csv
Read CSV file: /aiml/data/NoS_AW_Script/aw_caw.csv
Load Jieba Dict
True


In [ ]:
pd.concat(df_sorted)

### 单元测试

In [472]:
deal_str = Deal_Str()

Load Jieba Dict


In [473]:
input_str = '去配置二层组播实例基础业务'

In [474]:
deal_str.deal_chinese_like_str(input_str)

'Configure basic services of Layer 2 multicast instances'

In [424]:
ret_str = deal_str.regx.sub(' ', input_str);ret_str

'Check_igmpsnooping_portinfo"\r\n'

In [425]:
ret_str = ' '.join([x.strip() for x in ret_str.split() if "" != x]);ret_str

'Check_igmpsnooping_portinfo"'

In [426]:
ret_str = method_to_des(rm_new_line_toke(ret_str));ret_str

'Check igmpsnooping portinfo'

In [427]:
ret_str = ' '.join([wordnet_lemmatizer.lemmatize(x) for x in ret_str.split()]);ret_str

'Check igmpsnooping portinfo'

In [390]:
tmp = 'm'
tmp.isspace

## 收集专有词汇

In [178]:
with open('/tmp/filed_word_origin2.csv', 'r') as f:
    tmp = set(deal_str.deal_english_like_str(f.read()).split())

In [166]:
with open('/aiml/data/DevMind/06.SRC/01.TestCaseBot/02.TestAWBot/01.NMT_Train/nmt/preprocess/wordninja_words.txt', 'r') as f:
    dict_org = f.readlines()

In [177]:
dict_1 = [x for x in tmp if x not in dict_org and len(x) > 1]

In [181]:
dict_2 = [x for x in tmp if x not in dict_org and len(x) > 1]

In [183]:
dict_1.extend(dict_2)

In [189]:
tmp = 'S'
tmp.lower()

's'